In [44]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [45]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')

In [46]:
#Print some sample rows for investigating the data
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Here the RowNumber, CustomerID and Surname is irrelevent for predicting whether the clinet will exit or not so we just drop these columns (0,1,2). We also split the data into feature vectors X and the target vector y.  

In [47]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("Shape of X", X.shape)
print("Shape of y",y.shape)
print("First rown of X ",X[0,:])
print("First row of y ",y[0])

Shape of X (10000, 10)
Shape of y (10000,)
First rown of X  [619 'France' 'Female' 42 2 0.0 1 1 1 101348.88]
First row of y  1


We notice that the column Geography and Gender are categorical variables so we need to encode them into some integer variables. We do so using LabelEncoder fromt he sklearn.preprocessing package. 

In [48]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
print("Shape of X", X.shape)
print("Shape of y",y.shape)
print("After encoding the Geography (Column 2 now) and Gender Columns (Column 3 now)")
print("France as 0, Germany as 1, Spain as 2")
print("First row of X ",X[0,:])
print("First row of y ",y[0])

Shape of X (10000, 10)
Shape of y (10000,)
After encoding the Geography (Column 2 now) and Gender Columns (Column 3 now)
France as 0, Germany as 1, Spain as 2
First row of X  [619 0 0 42 2 0.0 1 1 1 101348.88]
First row of y  1


We now need to do OneHotEncoding for the Geography Column 2

In [49]:
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
print("Shape of X", X.shape)
print("Shape of y",y.shape)
print("First row of X ",X[0,:])
print("First row of y ",y[0])

Shape of X (10000, 12)
Shape of y (10000,)
First row of X  [  1.00000000e+00   0.00000000e+00   0.00000000e+00   6.19000000e+02
   0.00000000e+00   4.20000000e+01   2.00000000e+00   0.00000000e+00
   1.00000000e+00   1.00000000e+00   1.00000000e+00   1.01348880e+05]
First row of y  1


Now (1 0 0) is France, (0 1 0) is Germany and (0 0 1) is Spain. We can also exclude the first column since the last two are sufficient.

In [50]:
X = X[:, 1:]
print("Shape of X", X.shape)
print("Shape of y",y.shape)
print("First row of X ",X[0,:])
print("First row of y ",y[0])

Shape of X (10000, 11)
Shape of y (10000,)
First row of X  [  0.00000000e+00   0.00000000e+00   6.19000000e+02   0.00000000e+00
   4.20000000e+01   2.00000000e+00   0.00000000e+00   1.00000000e+00
   1.00000000e+00   1.00000000e+00   1.01348880e+05]
First row of y  1


In [51]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [52]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Build and train the ANN

In [53]:
import keras
from keras.models import Sequential
from keras.layers import Dense
ANN_classifier = Sequential()
ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
ANN_classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
ANN_classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

ANN_classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
8000/8000 [==============================] - 1s - loss: 0.5021 - acc: 0.7950     - ETA: 0s - loss: 0.5161 - acc: 0.79
Epoch 2/100
8000/8000 [==============================] - 0s - loss: 0.4307 - acc: 0.7960     
Epoch 3/100
8000/8000 [==============================] - 0s - loss: 0.4273 - acc: 0.7960     - ETA: 0s - loss: 0.4255 - acc: 0.79
Epoch 4/100
8000/8000 [==============================] - 0s - loss: 0.4238 - acc: 0.7960     - ETA: 0s - loss: 0.4214 - 
Epoch 5/100
8000/8000 [==============================] - 0s - loss: 0.4206 - acc: 0.8026     
Epoch 6/100
8000/8000 [==============================] - 0s - loss: 0.4183 - acc: 0.8229     
Epoch 7/100
8000/8000 [==============================] - 0s - loss: 0.4162 - acc: 0.8267     
Epoch 8/100
8000/8000 [==============================] - 0s - loss: 0.4145 - acc: 0.8285     
Epoch 9/100
8000/8000 [==============================] - 0s - loss: 0.4131 - acc: 0.8302     
Epoch 10/100
8000/8000 [==============================]

8000/8000 [==============================] - 0s - loss: 0.4002 - acc: 0.8349     
Epoch 86/100
8000/8000 [==============================] - 0s - loss: 0.4003 - acc: 0.8344     - ETA: 0s - loss: 0.4029 - acc: 
Epoch 87/100
8000/8000 [==============================] - 0s - loss: 0.4002 - acc: 0.8351     
Epoch 88/100
8000/8000 [==============================] - 0s - loss: 0.3999 - acc: 0.8356     
Epoch 89/100
8000/8000 [==============================] - 0s - loss: 0.4003 - acc: 0.8354     
Epoch 90/100
8000/8000 [==============================] - 0s - loss: 0.3997 - acc: 0.8349     
Epoch 91/100
8000/8000 [==============================] - 0s - loss: 0.4001 - acc: 0.8354     
Epoch 92/100
8000/8000 [==============================] - 0s - loss: 0.4000 - acc: 0.8362     
Epoch 93/100
8000/8000 [==============================] - 0s - loss: 0.3999 - acc: 0.8376     
Epoch 94/100
8000/8000 [==============================] - 0s - loss: 0.3999 - acc: 0.8346     
Epoch 95/100
8000/8000 [=======

In [8]:
# Predicting the Test set results
y_pred = ANN_classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [42]:
# Predicting a single new observation
"""Predict if the customer with the following informations will leave the bank:
Geography: France
Credit Score: 600
Gender: Male
Age: 40
Tenure: 3
Balance: 60000
Number of Products: 2
Has Credit Card: Yes
Is Active Member: Yes
Estimated Salary: 50000"""
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
print("The above customer leaves ", new_prediction)

The above customer leaves  [[False]]


In [43]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print("The confusion matrix is ",cm)

The confusion matrix is  [[1543   52]
 [ 257  148]]


Evaluating  the ANN

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
def build_classifier():
    ANN_classifier = Sequential()
    ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    ANN_classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    ANN_classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return ANN_classifier
ANN_classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = ANN_classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
variance = accuracies.std()

In [ ]:
print("Mean Accuracy ", mean)
print("Variance ",variance)

Improving the ANN using DropOut

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
ANN_classifier = Sequential()
ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
ANN_classifier.add(Dropout(p=0.1))
ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
ANN_classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
ANN_classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

ANN_classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Hyperparameter tuning using Grid Search

In [ ]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    ANN_classifier = Sequential()
    ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    ANN_classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    ANN_classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    ANN_classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
ANN_classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size': [25, 32],
              'epochs': [100, 500],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = ANN_classifier,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_

In [ ]:
print("Best parameters ",best_parameters)
print("Best accuracy ",best_accuracy)